In [52]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time


## Configurando driver e url


In [54]:
inicio_script = time.time()


driver = webdriver.Chrome()

url = 'https://www.amazon.com.br/hz/wishlist/ls/3TSX9V5QWRN9R?ref_=wl_share'

driver.get(url)

if driver.title: #se captar o título da página = sucesso
    print(f'Sucesso ao acessar: {driver.title}')
else:
    print(f'Erro ao acessar.')




Sucesso ao acessar: Amazon.com.br


## Carregando nome da lista de compras

In [56]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

nome_lista = soup.find('span', {'id': 'profile-list-name'})
if nome_lista:
    print(f'Lista: "{nome_lista.text}" carregada com sucesso!')
else:
    print(f'Nome da lista de compras não encontrada')


Lista: "Livros " carregada com sucesso!


## Funções

In [58]:
def carregar_pagina(driver,url):
    driver.get(url)
    return driver.page_source

def extrair_nome_lista(soup):
    nome_lista = soup.find('span', {'id': 'profile-list-name'})
    return nome_lista.text if nome_lista else 'Nome não encontrado'

def extrair_itens_lista(soup):
    itens = soup.find_all('li', {'class': 'g-item-sortable'})
    return [extrair_dados_itens(item) for item in itens]

def extrair_dados_itens(item):
    dados_item = {}
    dados_item['nome'] = item.find('a', {'class': 'a-link-normal'}).get('title', 'Nome não encontrado')
    dados_item['link'] = f"https://www.amazon.com.br{item.find('a', {'class': 'a-link-normal'}).get('href', '')}"
    dados_item['imagem'] = item.find('img').get('src', 'Imagem não encontrada') if item.find('img') else 'Imagem não encontrada'
    dados_item['preco'] = item.find('span', {'aria-hidden': 'true'}).text.strip() if item.find('span', {'aria-hidden': 'true'}) else 'Preço indisponível'
    return dados_item

In [59]:

nome_lista = extrair_nome_lista(soup)
print(f'Lista: {nome_lista} - Carregada com sucesso!')


itens_lista = extrair_itens_lista(soup)
if itens_lista:
    print(f'Itens encontrados na lista: {len(itens_lista)}\n')
    for item in itens_lista:
        for chave, valor in item.items():
            print(f'{chave.capitalize()}: {valor}')
        print(f'----------')
    
else:
    print(f'Itens da lista não encontrados')


Lista: Livros  - Carregada com sucesso!
Itens encontrados na lista: 10

Nome: Uma farsa de amor na Espanha (Sucesso do TikTok)
Link: https://www.amazon.com.br/dp/6555652977/?coliid=I32SK5IZ6IIRD&colid=3TSX9V5QWRN9R&psc=1&ref_=list_c_wl_lv_vv_lig_dp_it_im
Imagem: https://m.media-amazon.com/images/I/41sDZUdK+4L._SS135_.jpg
Preco: R$43,00
----------
Nome: Oração para Desaparecer
Link: https://www.amazon.com.br/dp/6559215709/?coliid=I24SDRDH3XO4ZW&colid=3TSX9V5QWRN9R&psc=1&ref_=list_c_wl_lv_vv_lig_dp_it_im
Imagem: https://m.media-amazon.com/images/I/311o3+q7YGL._SS135_.jpg
Preco: R$52,44
----------
Nome: A lista de convidados
Link: https://www.amazon.com.br/dp/6555601507/?coliid=I8NJ8PITN2O09&colid=3TSX9V5QWRN9R&psc=1&ref_=list_c_wl_lv_vv_lig_dp_it_im
Imagem: https://m.media-amazon.com/images/I/41OpQ0473IL._SS135_.jpg
Preco: R$38,67
----------
Nome: A última festa
Link: https://www.amazon.com.br/dp/8551005723/?coliid=ITZRRP6A4VU7B&colid=3TSX9V5QWRN9R&psc=1&ref_=list_c_wl_lv_vv_lig_dp_it_im

## Fechar navegador

In [61]:
driver.quit()


fim_script = time.time()
tempo_total_script = fim_script - inicio_script

print(f'Tempo total do script: {tempo_total_script:.2f} segundos')

Tempo total do script: 13.86 segundos
